In [1]:
import numpy as np
import pandas as pd

In [2]:
# data = pd.read_csv('../raw_data/100k_data.csv')

In [76]:
data_path = '../raw_data/100k_data.csv'
data = pd.read_csv(data_path)
# data = data.dropna()
df = data.dropna().sample(10000,random_state=0)

In [77]:
df.head(10)

,Unnamed: 0,id,time_stamp,title,Image_url,upvote_ratio,upvotes
26379,26379,nu260w,1.623034e+09,"Patriot Poms... Macie(all black pom) , pinning...",https://i.redd.it/41g91fvzbr371.png,1.00,5
46598,46598,g3c9su,1.587163e+09,Flora our golden retriever,https://i.redd.it/mj6cwtm1igt41.jpg,0.91,28
58211,58211,cwg55z,1.566968e+09,Does anybody know what kind of dog this is? We...,https://i.redd.it/b0zcwy7of4j31.jpg,0.95,15
13796,13796,sr2itc,1.644704e+09,My American Bulldog puppy,https://i.redd.it/7w5hzqvn6hh81.jpg,1.00,1
17481,17481,r7izrl,1.638484e+09,"Mac, chewer of the stick. Love life by the river.",https://www.reddit.com/gallery/r7izrl,0.91,15
23443,23443,oy6wi5,1.628126e+09,"My baby boy Gunner, a new father!",https://i.redd.it/8l48os1lwff71.jpg,0.94,13
23954,23954,orlr1p,1.627253e+09,Good girl had a stroke she lived 100 dog years...,https://i.redd.it/9dsiq2udsfd71.jpg,1.00,1
42796,42796,hjtt2d,1.593681e+09,Success,https://i.redd.it/0150bbtove851.jpg,0.95,108
15633,15633,ryus2n,1.641629e+09,lost my baby girl a year ago today. she got ra...,https://i.redd.it/bjv509ij7fa81.jpg,0.94,15
14393,14393,sifa5d,1.643776e+09,my dod in a christmas sweater 🥺🥺🥺🥺 do u like it?,https://www.reddit.com/gallery/sifa5d,1.00,1


In [4]:
data.title

0                                     i'm going to walmart
1        After suffering from 2 cherry eye look at how ...
2                                              Pets please
3                              Enjoying some Ice Cream :-)
4              What do you guys think of my grandma’s dog?
                               ...                        
99260                                  It's so green here!
99262            My vicious beast trying to catch his toy.
99263        Kiley, 14 years old and more regal than ever.
99264    My boys Kylo (left) and Peanut (right) love St...
99265                                              HEAVEN!
Name: title, Length: 72763, dtype: object

In [5]:
def count_len(text):
    text = text.split(' ')
    length = len(text)
    return length

In [6]:
df['title_len']=df['title'].apply(count_len)

In [7]:
data['title_len']=data['title'].apply(count_len)

In [8]:
df['title_len'].mean()

8.8879

## Packages

In [9]:
!pip install emot # package to process emoji

In [10]:
import re
import pickle
import string
from emot.emo_unicode import UNICODE_EMOJI
import nltk
import emoji

### Demojize

In [11]:
# function demojize(self, text):
#    emoji.demojize(text)

# function word2vec(self, text):
#    new_text = self.demojize(text)
#    my_vect = Word2Vec(new_test)
#    return my_vect

In [12]:
def demojize(text):
    clean_text = emoji.demojize(text)
    clean_text = clean_text.replace(':', '')
    clean_text = clean_text.replace('_', ' ')
    return clean_text

In [13]:
demojize('Python is 👍')

'Python is thumbs up'

In [14]:
df['title'] = df.title.apply(demojize)

## CLEANING

### For the Base Model, I decide to not consider the impact of special Punctuation like '?' and '!'
(strip + lowercase + numbers + punctuation/symbols)

In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
my_punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”’'

In [17]:
def basic_cleaning(sentence):
    sentence = sentence.lower()
    sentence = ''.join(char for char in sentence if not char.isdigit())
    for punctuation in my_punc:
        sentence = sentence.replace(punctuation, '')
    sentence = sentence.strip()
    return sentence

In [18]:
df['title'] = df.title.apply(basic_cleaning)

In [19]:
df['title']

26379    patriot poms macieall black pom  pinning her b...
46598                           flora our golden retriever
58211    does anybody know what kind of dog this is we ...
13796                            my american bulldog puppy
17481       mac chewer of the stick love life by the river
                               ...                        
82204                                         the best boy
86320                                         hello ladies
96940           swim stop mid hike in daylesford australia
39127                                   soakin up the rays
96839    rosie is having a busy day better dont disturb...
Name: title, Length: 10000, dtype: object

### Convert Emoji to Words

In [20]:
UNICODE_EMOJI

{'🥇': ':1st_place_medal:',
 '🥈': ':2nd_place_medal:',
 '🥉': ':3rd_place_medal:',
 '🆎': ':AB_button_(blood_type):',
 '🏧': ':ATM_sign:',
 '🅰': ':A_button_(blood_type):',
 '🇦🇫': ':Afghanistan:',
 '🇦🇱': ':Albania:',
 '🇩🇿': ':Algeria:',
 '🇦🇸': ':American_Samoa:',
 '🇦🇩': ':Andorra:',
 '🇦🇴': ':Angola:',
 '🇦🇮': ':Anguilla:',
 '🇦🇶': ':Antarctica:',
 '🇦🇬': ':Antigua_&_Barbuda:',
 '♒': ':Aquarius:',
 '🇦🇷': ':Argentina:',
 '♈': ':Aries:',
 '🇦🇲': ':Armenia:',
 '🇦🇼': ':Aruba:',
 '🇦🇨': ':Ascension_Island:',
 '🇦🇺': ':Australia:',
 '🇦🇹': ':Austria:',
 '🇦🇿': ':Azerbaijan:',
 '🔙': ':BACK_arrow:',
 '🅱': ':B_button_(blood_type):',
 '🇧🇸': ':Bahamas:',
 '🇧🇭': ':Bahrain:',
 '🇧🇩': ':Bangladesh:',
 '🇧🇧': ':Barbados:',
 '🇧🇾': ':Belarus:',
 '🇧🇪': ':Belgium:',
 '🇧🇿': ':Belize:',
 '🇧🇯': ':Benin:',
 '🇧🇲': ':Bermuda:',
 '🇧🇹': ':Bhutan:',
 '🇧🇴': ':Bolivia:',
 '🇧🇦': ':Bosnia_&_Herzegovina:',
 '🇧🇼': ':Botswana:',
 '🇧🇻': ':Bouvet_Island:',
 '🇧🇷': ':Brazil:',
 '🇮🇴': ':British_Indian_Ocean_Territory:',
 '🇻🇬': ':British_Vir

In [21]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()), text)
    return text

In [13]:
df['title'] = df['title'].apply(convert_emojis)

error: nothing to repeat at position 1

### Remove Stopwords

In [22]:
from nltk.corpus import stopwords

In [23]:
STOPWORDS = set(stopwords.words('english'))

In [24]:
'me' in STOPWORDS

True

In [25]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [26]:
df['no_stopword']=df['title']

In [27]:
df['no_stopword'] = df['no_stopword'].apply(remove_stopwords)

In [28]:
remove_stopwords('can someone tell me how she')

'someone tell'

In [29]:
# how to deal with # (tags); how to deal with 2 emojis # how to deal with !&?; Chat Words Conversion e.g. wtf/lmk

### Tokenize

In [30]:
from nltk.tokenize import word_tokenize

In [31]:
df['no_stopword'] = df.no_stopword.apply(word_tokenize)

In [32]:
df['no_stopword'].head(20)

26379    [patriot, poms, macieall, black, pom, pinning,...
46598                           [flora, golden, retriever]
58211    [anybody, know, kind, dog, found, stray, exact...
13796                           [american, bulldog, puppy]
17481              [mac, chewer, stick, love, life, river]
23443                     [baby, boy, gunner, new, father]
23954    [good, girl, stroke, lived, dog, years, hope, ...
42796                                            [success]
15633    [lost, baby, girl, year, ago, today, got, ran,...
14393    [dod, christmas, sweater, pleading, facepleadi...
85833                          [lbs, pure, love, devotion]
27951                    [leia, great, time, friend, pier]
98173              [sadie, took, first, boat, ride, today]
88726                             [xanders, rd, christmas]
70956    [elderly, dog, rubella, thankful, longer, live...
11373       [meet, sherlock, adorable, boy, melted, heart]
31117                [newest, addition, big, family, nov

### Lemmatize

In [33]:
from nltk.stem import WordNetLemmatizer

def lemma(text):
    lemmatizer = WordNetLemmatizer() # Instantiate lemmatizer
    lemmatized_n = [lemmatizer.lemmatize(word,pos='n') for word in text]
#     print(lemmatized_n)
    lemmatized_v = [lemmatizer.lemmatize(word,pos='v') for word in lemmatized_n] 
#     print(lemmatized_v)
#     cleaned_sentence = ' '.join(word for word in lemmatized_v)
#     return cleaned_sentence
    return lemmatized_v

In [34]:
df['lemmatize'] = df['no_stopword']

In [35]:
df['no_stopword']

26379    [patriot, poms, macieall, black, pom, pinning,...
46598                           [flora, golden, retriever]
58211    [anybody, know, kind, dog, found, stray, exact...
13796                           [american, bulldog, puppy]
17481              [mac, chewer, stick, love, life, river]
                               ...                        
82204                                          [best, boy]
86320                                      [hello, ladies]
96940       [swim, stop, mid, hike, daylesford, australia]
39127                                       [soakin, rays]
96839            [rosie, busy, day, better, dont, disturb]
Name: no_stopword, Length: 10000, dtype: object

In [36]:
df['lemmatize'] = df['lemmatize'].apply(lemma)

In [37]:
lemma('someone, tell, got, lid')

['s',
 'o',
 'm',
 'e',
 'o',
 'n',
 'e',
 ',',
 ' ',
 't',
 'e',
 'l',
 'l',
 ',',
 ' ',
 'g',
 'o',
 't',
 ',',
 ' ',
 'l',
 'i',
 'd']

In [38]:
df['lemmatize'].head(20)

26379    [patriot, pom, macieall, black, pom, pin, baby...
46598                           [flora, golden, retriever]
58211    [anybody, know, kind, dog, find, stray, exactl...
13796                           [american, bulldog, puppy]
17481              [mac, chewer, stick, love, life, river]
23443                     [baby, boy, gunner, new, father]
23954    [good, girl, stroke, live, dog, year, hope, sh...
42796                                            [success]
15633    [lose, baby, girl, year, ago, today, get, run,...
14393    [dod, christmas, sweater, plead, facepleading,...
85833                           [lb, pure, love, devotion]
27951                    [leia, great, time, friend, pier]
98173              [sadie, take, first, boat, ride, today]
88726                             [xanders, rd, christmas]
70956    [elderly, dog, rubella, thankful, longer, live...
11373         [meet, sherlock, adorable, boy, melt, heart]
31117                [newest, addition, big, family, nov

Count the length of each title (Need to split all the words and emoji beforehand)

In [149]:
X_lemma = df['lemmatize']

In [59]:
df.head(100)

,Unnamed: 0,id,time_stamp,title,Image_url,upvote_ratio,upvotes,no_stopword,lemmatize
26379,26379,nu260w,1.623034e+09,patriot poms macieall black pom pinning her b...,https://i.redd.it/41g91fvzbr371.png,1.00,5,"[patriot, poms, macieall, black, pom, pinning,...","[patriot, pom, macieall, black, pom, pin, baby..."
46598,46598,g3c9su,1.587163e+09,flora our golden retriever,https://i.redd.it/mj6cwtm1igt41.jpg,0.91,28,"[flora, golden, retriever]","[flora, golden, retriever]"
58211,58211,cwg55z,1.566968e+09,does anybody know what kind of dog this is we ...,https://i.redd.it/b0zcwy7of4j31.jpg,0.95,15,"[anybody, know, kind, dog, found, stray, exact...","[anybody, know, kind, dog, find, stray, exactl..."
13796,13796,sr2itc,1.644704e+09,my american bulldog puppy,https://i.redd.it/7w5hzqvn6hh81.jpg,1.00,1,"[american, bulldog, puppy]","[american, bulldog, puppy]"
17481,17481,r7izrl,1.638484e+09,mac chewer of the stick love life by the river,https://www.reddit.com/gallery/r7izrl,0.91,15,"[mac, chewer, stick, love, life, river]","[mac, chewer, stick, love, life, river]"
...,...,...,...,...,...,...,...,...,...
78789,78789,8y943f,1.531397e+09,no matter how many times i put the pillows on ...,https://i.redd.it/mf8u0abqai911.jpg,0.85,8,"[matter, many, times, put, pillows, sofa, alwa...","[matter, many, time, put, pillow, sofa, always..."
76207,76207,9c8umb,1.535856e+09,brought our new baby home from the shelter nee...,https://i.redd.it/e2zfnednmqj11.jpg,0.97,83,"[brought, new, baby, home, shelter, need, name...","[bring, new, baby, home, shelter, need, name, ..."
876,876,yh224t,1.667096e+09,a very sweet boy from the shelter i work at re...,https://i.redd.it/g9xlr3xr8ww91.jpg,0.99,60,"[sweet, boy, shelter, work, red, heart]","[sweet, boy, shelter, work, red, heart]"
18248,18248,quew0p,1.636978e+09,guess what shes looking at whisper not a treat,https://i.redd.it/g8nda4tq1rz71.jpg,1.00,380,"[guess, shes, looking, whisper, treat]","[guess, shes, look, whisper, treat]"


In [ ]:
# cannot deal with abbreviation; doin; cannot seperate emoji

### Tokenizer

In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(df.lemmatize)
df['tokenized']=tk.texts_to_sequences(df.lemmatize)

In [40]:
df.head(30)

,Unnamed: 0,id,time_stamp,title,Image_url,upvote_ratio,upvotes,title_len,no_stopword,lemmatize,tokenized
26379,26379,nu260w,1.623034e+09,patriot poms macieall black pom pinning her b...,https://i.redd.it/41g91fvzbr371.png,1.00,5,16,"[patriot, poms, macieall, black, pom, pinning,...","[patriot, pom, macieall, black, pom, pin, baby...","[3422, 906, 3423, 192, 906, 3424, 30, 11, 1152..."
46598,46598,g3c9su,1.587163e+09,flora our golden retriever,https://i.redd.it/mj6cwtm1igt41.jpg,0.91,28,4,"[flora, golden, retriever]","[flora, golden, retriever]","[3425, 166, 369]"
58211,58211,cwg55z,1.566968e+09,does anybody know what kind of dog this is we ...,https://i.redd.it/b0zcwy7of4j31.jpg,0.95,15,35,"[anybody, know, kind, dog, found, stray, exact...","[anybody, know, kind, dog, find, stray, exactl...","[691, 50, 357, 1, 60, 513, 813, 153, 55, 1, 51..."
13796,13796,sr2itc,1.644704e+09,my american bulldog puppy,https://i.redd.it/7w5hzqvn6hh81.jpg,1.00,1,4,"[american, bulldog, puppy]","[american, bulldog, puppy]","[487, 305, 17]"
17481,17481,r7izrl,1.638484e+09,mac chewer of the stick love life by the river,https://www.reddit.com/gallery/r7izrl,0.91,15,10,"[mac, chewer, stick, love, life, river]","[mac, chewer, stick, love, life, river]","[1533, 3426, 127, 4, 49, 569]"
23443,23443,oy6wi5,1.628126e+09,my baby boy gunner a new father,https://i.redd.it/8l48os1lwff71.jpg,0.94,13,7,"[baby, boy, gunner, new, father]","[baby, boy, gunner, new, father]","[30, 5, 1029, 18, 1153]"
23954,23954,orlr1p,1.627253e+09,good girl had a stroke she lived dog years ho...,https://i.redd.it/9dsiq2udsfd71.jpg,1.00,1,19,"[good, girl, stroke, lived, dog, years, hope, ...","[good, girl, stroke, live, dog, year, hope, sh...","[12, 11, 1867, 200, 1, 7, 204, 28, 214, 603, 295]"
42796,42796,hjtt2d,1.593681e+09,success,https://i.redd.it/0150bbtove851.jpg,0.95,108,1,[success],[success],[1534]
15633,15633,ryus2n,1.641629e+09,lost my baby girl a year ago today she got ran...,https://i.redd.it/bjv509ij7fa81.jpg,0.94,15,29,"[lost, baby, girl, year, ago, today, got, ran,...","[lose, baby, girl, year, ago, today, get, run,...","[167, 30, 11, 7, 77, 20, 3, 156, 24, 8, 193, 8..."
14393,14393,sifa5d,1.643776e+09,my dod in a christmas sweater pleading faceple...,https://www.reddit.com/gallery/sifa5d,1.00,1,11,"[dod, christmas, sweater, pleading, facepleadi...","[dod, christmas, sweater, plead, facepleading,...","[3427, 85, 1030, 488, 1031, 1031, 1031, 2, 83,..."


In [170]:
tk.word_index

{'dog': 1,
 'get': 2,
 'boy': 3,
 'love': 4,
 'old': 5,
 'year': 6,
 'day': 7,
 'delete': 8,
 'user': 9,
 'happy': 10,
 'good': 11,
 'girl': 12,
 'little': 13,
 'like': 14,
 'best': 15,
 'look': 16,
 'today': 17,
 'puppy': 18,
 'new': 19,
 'friend': 20,
 'meet': 21,
 'take': 22,
 'time': 23,
 'shes': 24,
 'he': 25,
 'one': 26,
 'month': 27,
 'pup': 28,
 'go': 29,
 'first': 30,
 'picture': 31,
 'think': 32,
 'say': 33,
 'im': 34,
 'baby': 35,
 'want': 36,
 'week': 37,
 'enjoy': 38,
 'doggo': 39,
 'sleep': 40,
 'sweet': 41,
 'make': 42,
 'cute': 43,
 'walk': 44,
 'name': 45,
 'breed': 46,
 'home': 47,
 'mix': 48,
 'life': 49,
 'know': 50,
 'smile': 51,
 'miss': 52,
 'ago': 53,
 'play': 54,
 'ever': 55,
 'rescue': 56,
 'face': 57,
 'see': 58,
 'big': 59,
 '❤️': 60,
 'always': 61,
 'beautiful': 62,
 'guy': 63,
 'still': 64,
 'help': 65,
 'two': 66,
 'find': 67,
 'dont': 68,
 'much': 69,
 'morning': 70,
 'eye': 71,
 'snow': 72,
 'work': 73,
 'birthday': 74,
 'buddy': 75,
 'sit': 76,
 'bed':

### Preprocess Whole Data

In [79]:
data['basic_cleaning'] = data.title.apply(basic_cleaning)

In [80]:
data['no_stopword'] = data['basic_cleaning'].apply(remove_stopwords)

In [81]:
data['no_stopword'] = data.no_stopword.apply(word_tokenize)

In [82]:
data['lemmatize'] = data['no_stopword'].apply(lemma)

### Word2Vector (Alternative to tokenize)

#### Pre-trained model glove-twitter-25

In [78]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load('glove-twitter-25')
glove_vectors.most_similar('twitter')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


KeyboardInterrupt: 

#### My model of whole dataset

In [88]:
from gensim.models import Word2Vec
word2vec = Word2Vec(sentences=data['lemmatize'], vector_size=60, min_count=10, window=4)
# how to select vector_size?

In [63]:
word2vec # trained base on whole dataset

In [64]:
word2vec.wv['dog']

array([ 0.07263631, -0.5084555 ,  1.2883906 ,  0.37998876,  0.12062747,
       -1.2239171 ,  0.8351777 ,  0.52726537, -0.1113221 , -0.35906237,
        0.550001  , -0.8102392 ,  0.11292174, -0.12290344, -0.9436515 ,
        0.79621   , -0.27030084, -0.7467119 , -1.0191675 , -0.45663548,
       -0.04955515,  0.71441954,  0.5066333 ,  0.05662412,  0.12155697,
        0.3055172 , -0.26237327,  0.21152979, -0.757567  , -0.25389147,
       -0.52203673, -0.0917214 ,  0.0568262 , -0.28411174,  0.08060788,
        0.6304348 ,  0.43507713, -0.18436636,  0.34251413, -0.2985299 ,
       -0.48771605,  0.26464522,  0.03484035,  0.24232462,  0.0308924 ,
       -0.17314051, -0.2871781 ,  0.53986764, -0.19218914,  1.033735  ],
      dtype=float32)

In [94]:
word2vec.wv.most_similar('four', topn=15)

[('six', 0.9544544219970703),
 ('three', 0.9426869750022888),
 ('nearly', 0.9316259622573853),
 ('ten', 0.9184859991073608),
 ('five', 0.918082058429718),
 ('pound', 0.9090571403503418),
 ('nine', 0.9083735942840576),
 ('almost', 0.9048264026641846),
 ('ish', 0.891463577747345),
 ('difference', 0.8830099701881409),
 ('couple', 0.8811892867088318),
 ('le', 0.8797865509986877),
 ('seven', 0.8744874000549316),
 ('litter', 0.8736796379089355),
 ('weigh', 0.8708064556121826)]

#### Pre-trained model 'glove-twitter-200'

In [49]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [51]:
twitter200_corpus = api.load('glove-twitter-200')
twitter200_corpus.most_similar('twitter')

KeyboardInterrupt: 

In [76]:
twitter200_corpus.similarity('tree', 'leaf')

NameError: name 'twitter200_corpus' is not defined

In [234]:
twitter200_corpus.most_similar('dog', topn=10)

[('dogs', 0.8483485579490662),
 ('cat', 0.832430362701416),
 ('puppy', 0.7890411019325256),
 ('pet', 0.7524258494377136),
 ('cats', 0.7141677141189575),
 ('horse', 0.6632055640220642),
 ('animal', 0.657815158367157),
 ('kitten', 0.6441184878349304),
 ('little', 0.632893979549408),
 ('kid', 0.6253357529640198)]

### embedding

### Padding

In [77]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(, dtype='float32', padding='post', maxlen=60)

SyntaxError: invalid syntax (383221832.py, line 2)

In [138]:
X_train.shape

(10000, 60)